In [10]:
import pandas
from tinydb import TinyDB
import numpy

data = pandas.DataFrame(TinyDB('data/db/db.json').table('games').all())
print(data.head())
print(data.shape)

    app_id                               name          category  \
0   536280                     Disintegration      Action Games   
1   982800                        Moon Tycoon  Simulation Games   
2   767960                           Math RTS    Strategy Games   
3  1368670  Beach Volleyball Competition 2020      Sports Games   
4   977800          At Home Alone - Reward 5$   Adventure Games   

                                                tags  \
0  [FPS, PvE, Vehicular Combat, RTS, Shooter, Str...   
1  [Strategy, Simulation, Indie, Management, Sing...   
2  [Strategy, Indie, Education, RTS, Real-Time, S...   
3  [Sports, Action, Arcade, Character Action Game...   
4  [Strategy, Adventure, Free to Play, RPG, Indie...   

                                     reviews  \
0  {'overall': {'rating': 61, 'count': 120}}   
1           {'overall': 'need_more_reviews'}   
2           {'overall': 'need_more_reviews'}   
3           {'overall': 'need_more_reviews'}   
4           {'overal

In [11]:
# Replacing placeholder value in review data with NaN:
data.loc[(data['reviews'] == {'overall': 'need_more_reviews'}),'reviews'] = numpy.nan
# Splitting review data into overall and recent:
data = pandas.concat([data.drop(['reviews'], axis=1), data['reviews'].apply(pandas.Series)], axis=1)
del data[0]
# Getting rid of recent review data as it is not needed:
del data["recent"]
print(data.head())

    app_id                               name          category  \
0   536280                     Disintegration      Action Games   
1   982800                        Moon Tycoon  Simulation Games   
2   767960                           Math RTS    Strategy Games   
3  1368670  Beach Volleyball Competition 2020      Sports Games   
4   977800          At Home Alone - Reward 5$   Adventure Games   

                                                tags  \
0  [FPS, PvE, Vehicular Combat, RTS, Shooter, Str...   
1  [Strategy, Simulation, Indie, Management, Sing...   
2  [Strategy, Indie, Education, RTS, Real-Time, S...   
3  [Sports, Action, Arcade, Character Action Game...   
4  [Strategy, Adventure, Free to Play, RPG, Indie...   

                                         playtracker  \
0  {'popularity': 2, 'estimated_players': 159000,...   
1                                                NaN   
2                                                NaN   
3                                   

In [12]:
# Splitting overall review data into two columns:
data = pandas.concat([data.drop(['overall'], axis=1), data['overall'].apply(pandas.Series)], axis=1)
del data[0]
data.rename(columns = {'count':'review_count'}, inplace = True)
print(data.head())

    app_id                               name          category  \
0   536280                     Disintegration      Action Games   
1   982800                        Moon Tycoon  Simulation Games   
2   767960                           Math RTS    Strategy Games   
3  1368670  Beach Volleyball Competition 2020      Sports Games   
4   977800          At Home Alone - Reward 5$   Adventure Games   

                                                tags  \
0  [FPS, PvE, Vehicular Combat, RTS, Shooter, Str...   
1  [Strategy, Simulation, Indie, Management, Sing...   
2  [Strategy, Indie, Education, RTS, Real-Time, S...   
3  [Sports, Action, Arcade, Character Action Game...   
4  [Strategy, Adventure, Free to Play, RPG, Indie...   

                                         playtracker  review_count  rating  
0  {'popularity': 2, 'estimated_players': 159000,...         120.0    61.0  
1                                                NaN           NaN     NaN  
2                            

In [13]:
# Splitting playtracker into multiple columns:
data = pandas.concat([data.drop(['playtracker'], axis=1), data['playtracker'].apply(pandas.Series)], axis=1)
del data[0]
print(data.head())

    app_id                               name          category  \
0   536280                     Disintegration      Action Games   
1   982800                        Moon Tycoon  Simulation Games   
2   767960                           Math RTS    Strategy Games   
3  1368670  Beach Volleyball Competition 2020      Sports Games   
4   977800          At Home Alone - Reward 5$   Adventure Games   

                                                tags  review_count  rating  \
0  [FPS, PvE, Vehicular Combat, RTS, Shooter, Str...         120.0    61.0   
1  [Strategy, Simulation, Indie, Management, Sing...           NaN     NaN   
2  [Strategy, Indie, Education, RTS, Real-Time, S...           NaN     NaN   
3  [Sports, Action, Arcade, Character Action Game...           NaN     NaN   
4  [Strategy, Adventure, Free to Play, RPG, Indie...           NaN     NaN   

   average_playtime  estimated_active_players  estimated_players  popularity  
0               0.6                   25000.0    

In [14]:
# Splitting the dataset into two to make operations for tags easier:
game_info = data.copy()
del game_info['tags']
tags = pandas.DataFrame(data['tags'])
# Creating a column for each tag:
tags = pandas.concat([tags.drop(['tags'], axis=1), tags['tags'].apply(pandas.Series)], axis=1)
tags = pandas.get_dummies(tags)
tags.rename(columns=lambda x: x.split("_")[1], inplace=True)
tags = tags.groupby(level=0, axis=1).sum()
print(game_info.head())
print(tags.head())

    app_id                               name          category  review_count  \
0   536280                     Disintegration      Action Games         120.0   
1   982800                        Moon Tycoon  Simulation Games           NaN   
2   767960                           Math RTS    Strategy Games           NaN   
3  1368670  Beach Volleyball Competition 2020      Sports Games           NaN   
4   977800          At Home Alone - Reward 5$   Adventure Games           NaN   

   rating  average_playtime  estimated_active_players  estimated_players  \
0    61.0               0.6                   25000.0           159000.0   
1     NaN               NaN                       NaN                NaN   
2     NaN               NaN                       NaN                NaN   
3     NaN               NaN                       NaN                NaN   
4     NaN               NaN                       NaN                NaN   

   popularity  
0         2.0  
1         NaN  
2       

In [15]:
# Merging the tables back together:
games = game_info.merge(tags, left_index=True, right_index=True)
print(games.head())
print(games.shape)

    app_id                               name          category  review_count  \
0   536280                     Disintegration      Action Games         120.0   
1   982800                        Moon Tycoon  Simulation Games           NaN   
2   767960                           Math RTS    Strategy Games           NaN   
3  1368670  Beach Volleyball Competition 2020      Sports Games           NaN   
4   977800          At Home Alone - Reward 5$   Adventure Games           NaN   

   rating  average_playtime  estimated_active_players  estimated_players  \
0    61.0               0.6                   25000.0           159000.0   
1     NaN               NaN                       NaN                NaN   
2     NaN               NaN                       NaN                NaN   
3     NaN               NaN                       NaN                NaN   
4     NaN               NaN                       NaN                NaN   

   popularity  1980s  ...  Web Publishing  Well-Written 

In [16]:
# Checking if the tags seem right for tag "Strategy":
print(games["Strategy"].head())
# They seem to be right.

0    1
1    1
2    1
3    0
4    1
Name: Strategy, dtype: uint8


In [19]:
# Checking for nas:
pandas.set_option('display.max_rows', None)
print(games.isnull().sum())
# For more than half of the games review data is missing. For playtracker data even more.

app_id                                   0
name                                     0
category                                 0
review_count                         43741
rating                               43741
average_playtime                     56230
estimated_active_players             56230
estimated_players                    56230
popularity                           56230
1980s                                    0
1990's                                   0
2.5D                                     0
2D                                       0
2D Fighter                               0
2D Platformer                            0
360 Video                                0
3D                                       0
3D Fighter                               0
3D Platformer                            0
3D Vision                                0
4 Player Local                           0
4X                                       0
6DOF                                     0
ATV        